# WindFarmer API demo: AnnualEnergyProduction of The Bowl Wind Farm
The WindFarmer API is a web API. You can call it from any coding language, or use tools like Postman.

There is an OpenAPI definition which provides
documentation, and allows client code to be generated.

From python you can call the API directly, using `urllib3` or `requests`.

This script will compute AEP, wakes and blockage for a small wind farm using, firstly, the eddy viscosity wake model and BEET blockage model, then CFD.ML v1, then compare results from the two approaches. 

## Using the API directly
First, import the necessary modules

In [1]:
import os
import requests
import json
import time
from matplotlib import pyplot as plt
import pandas as pd
from copy import deepcopy

To access the API you need a authorization token. This should be kept secure - and not added to source control, so I'm getting it from an environment variable.

The token should be passed as an Authorization header. We also need to set the `Content-Type` to let the API know that we're sending JSON data.

In [2]:
api_url = 'https://windfarmer.dnv.com/api/v2/'
auth_token = os.environ['WINDFARMER_ACCESS_KEY']
headers = {
    'Authorization': f'Bearer {auth_token}',
    'Content-Type': 'application/json'
}

### Call `Status`
Try calling the `Status` endpoint. This verifies that you can access the API and that your token is valid.

If you get errors try `pip install pip-system-certs` for your environment. That allows python to use the Windows certificates store.

In [3]:
response = requests.get(api_url + 'Status', headers = headers)
print(f'Response from Status: {response.status_code}')
print(response.text)

### Hypothetical wind farm in flat terrain: `The Bowl`

The json containing inputs necessary to run an energy calculation in the cloud can be easiy generated in DNV's WindFarmer:Analyst, or can be compiled manually. 

The formal API spec is available here: https://windfarmer.dnv.com/documents 

In [4]:
with open('..\..\..\DemoData\TheBowl\TheBowl.json') as f:
    json_string = f.read()
    input_data = json.loads(json_string)

print(json_string[0:500] +  '...')

Let's plot the layout

In [5]:
fig1, ax = plt.subplots(figsize=(10,7))
for wt in input_data['windFarms'][0]['turbines']:
    ax.scatter(wt['location']['easting_m'], wt['location']['northing_m'], c='b')
    ax.set_xlabel("Easting [m]")
    ax.set_ylabel("Northing [m]")
    ax.set_aspect('equal', adjustable='box')
    ax.annotate(wt['name'],[wt['location']['easting_m'], wt['location']['northing_m']] )
ax.set_title('The Bowl - a hypothetical wind farm')
southern_edge = ['T95', 'T104', 'T99', 'T107', 'T102', 'T110', 'T97', 'T105', 'T100', 'T108', 'T103', 'T98', 'T106', 'T101', 'T109', 'T96']


In [6]:
print("The turbine type is: " + input_data['turbineModels'][0]['id'])

The wind:

![windrose](images/windrose.png)

![fd](images/fd.png)

The turbine interaction model settings:

In [7]:
#change blockage application method relative to the default from the file
input_data['energyEfficienciesSettings']['blockageModel']['beet']['blockageCorrectionApplicationMethod'] = "OnWindSpeed" 
# and print the final settings
print('wake calculation type: ' + input_data['energyEfficienciesSettings']['wakeModel']['wakeModelType'])
print('\twake model settings: ')
for setting, value in input_data['energyEfficienciesSettings']['wakeModel']['eddyViscosity'].items():
    print('\t\t'+str(setting)+': '+str(value))
print('blockage calculation type: ' + input_data['energyEfficienciesSettings']['blockageModel']['blockageModelType'])
print('\tblockage model settings: ')
for setting, value in input_data['energyEfficienciesSettings']['blockageModel']['beet'].items():
    print('\t\t'+str(setting)+': '+str(value))


### Call `AnnualEnergyProduction` endpoint

Send the input data to the Annual Energy Production calculation

In [8]:
start = time.time()
response = requests.post(
    api_url + 'AnnualEnergyProduction', 
    headers=headers,
    json = input_data)
    
print(f'Response {response.status_code} - {response.reason} in {time.time() - start:.2f}s')
# Print the error detail if we haven't receieved a 200 OK response
if response.status_code != 200:
    print(json.loads(response.content)['detail'])

### Inspect and plot the results

In [9]:
#extract the most important farm-level numbers
result = json.loads(response.content)
full_aep_MWh_per_year = float(result['windFarmAepOutputs'][0]['fullAnnualEnergyYield_MWh_per_year'])
beet_result = float(result['weightedBlockageEfficiency'])
print(f'Wake Affected Annual Energy Production = {full_aep_MWh_per_year:.1f} MWh / year')

#plot  per-turbine yield
per_turbine_results = pd.DataFrame.from_dict(result['windFarmAepOutputs'][0]['turbineResults'])
per_turbine_results['easting'] = per_turbine_results['turbineLocation'].map(lambda x: dict(x)['easting_m'])
per_turbine_results['northing'] = per_turbine_results['turbineLocation'].map(lambda x: dict(x)['northing_m'])
fig, ax = plt.subplots(figsize=(10,7))
im = ax.scatter(per_turbine_results['easting'], per_turbine_results['northing'], c=per_turbine_results['fullAnnualYield_MWh_per_year'], cmap='BuGn')
ax.set_title("Per turbine energy production (affected by wakes only) [MWh/yr]")
ax.set_xlabel("Easting [m]")
ax.set_ylabel("Northing [m]")
ax.set_aspect('equal', adjustable='box')
fig.colorbar(im)

### How to capture the `impact of blockage on a per-turbine basis`?

Let's modify the settings and switch to `CFD.ML`, DNV's new turbine interaction model capable of capturing wakes & blockage together, on a per-turbine basis.

In [10]:
input_data_cfdml = deepcopy(input_data)
input_data_cfdml['energyEfficienciesSettings']['wakeModel']['wakeModelType'] = 'CFDML'
input_data_cfdml["energyEfficienciesSettings"]["wakeModel"]["cfdml"]["gnnVersion"] = "1.1" # This calculator is only for version 1
# CFD.ML models wake&blockage together, so we always compute CFD.ML blockage if computing CFD.ML wakes, and it is best computed OnWindSpeed
input_data_cfdml['energyEfficienciesSettings']['blockageModel']['blockageModelType'] = "CFDML"
input_data_cfdml['energyEfficienciesSettings']['blockageModel']['cfdml']['blockageCorrectionApplicationMethod'] = "OnWindSpeed"
print("CFDML settings: " + str(input_data['energyEfficienciesSettings']['wakeModel']['cfdml']))

input_data_no_interactions = deepcopy(input_data)
input_data_no_interactions['energyEfficienciesSettings']['blockageModel']['blockageModelType'] = "NoBlockageModel" 
input_data_no_interactions['energyEfficienciesSettings']['wakeModel']['wakeModelType'] = 'NoWakeModel'

### Call the `AnnualEnergyProduction` API again

In [11]:
start = time.time()
response_cfdml = requests.post(
    api_url + 'AnnualEnergyProduction', 
    headers=headers,
    json = input_data_cfdml)
    
print(f'Response {response_cfdml.status_code} - {response_cfdml.reason} in {time.time() - start:.2f}s')
# Print the error detail if we haven't receieved a 200 OK response
if response_cfdml.status_code != 200:
    print(json.loads(response_cfdml.content)['detail'])

start = time.time()
response_no_interactions = requests.post(
    api_url + 'AnnualEnergyProduction', 
    headers=headers,
    json = input_data_no_interactions)
    
print(f'Response {response_no_interactions.status_code} - {response_no_interactions.reason} in {time.time() - start:.2f}s')
# Print the error detail if we haven't receieved a 200 OK response
if response_no_interactions.status_code != 200:
    print(json.loads(response_no_interactions.content)['detail'])

### Inspect & plot the results

In [ ]:
#extract the most important farm-level numbers
result_cfdml = json.loads(response_cfdml.content)
full_aep_MWh_per_year = float(result_cfdml['windFarmAepOutputs'][0]['fullAnnualEnergyYield_MWh_per_year'])
print(f'Wake and Blockage Affected Annual Energy Production = {full_aep_MWh_per_year:.1f} MWh / year')

per_turbine_results = pd.DataFrame.from_dict(result_cfdml['windFarmAepOutputs'][0]['turbineResults'])
per_turbine_results.set_index('turbineName', drop=True, inplace=True)
per_turbine_results['easting'] = per_turbine_results['turbineLocation'].map(lambda x: dict(x)['easting_m'])
per_turbine_results['northing'] = per_turbine_results['turbineLocation'].map(lambda x: dict(x)['northing_m'])
fig, ax = plt.subplots(figsize=(10,4))
im = ax.scatter(per_turbine_results['easting'], per_turbine_results['northing'], c=per_turbine_results['fullAnnualYield_MWh_per_year'], cmap='BuGn')
ax.set_title("Per turbine energy production (blockage & wakes affected) [MWh/yr]")
ax.set_xlabel("Easting [m]")
ax.set_ylabel("Northing [m]")
ax.set_aspect('equal', adjustable='box')
fig.colorbar(im)

### Windspeeds predicted by both models

In [ ]:
# EV + LWF
waked_windspeeds = pd.DataFrame(result['windFarmAepOutputs'][0]['turbineFlowAndPerformanceMatricesWithMastBinning'])
waked_windspeeds.set_index('turbineName',drop=True, inplace=True)
waked_windspeeds['wakedWindSpeed_at_8_m_per_s_at_180_deg'] = waked_windspeeds['wakedWindSpeed_m_per_s'].map(lambda x: round(x[90][8],3))
waked_windspeeds['wakedWindSpeed_at_8_m_per_s_at_90_deg'] = waked_windspeeds['wakedWindSpeed_m_per_s'].map(lambda x: round(x[45][8],3))
# CFD.ML
waked_windspeeds_cfdml = pd.DataFrame(result_cfdml['windFarmAepOutputs'][0]['turbineFlowAndPerformanceMatricesWithMastBinning'])
waked_windspeeds_cfdml.set_index('turbineName',drop=True, inplace=True)
waked_windspeeds_cfdml['wakedWindSpeed_at_8_m_per_s_at_180_deg'] = waked_windspeeds_cfdml['wakedWindSpeed_m_per_s'].map(lambda x: x[90][8])
waked_windspeeds_cfdml['wakedWindSpeed_at_8_m_per_s_at_90_deg'] = waked_windspeeds_cfdml['wakedWindSpeed_m_per_s'].map(lambda x: x[45][8])
# Plot
fix, ax = plt.subplots(figsize=(7,4))
waked_windspeeds.loc[southern_edge]['wakedWindSpeed_at_8_m_per_s_at_180_deg'].plot(marker='o', ax=ax)
waked_windspeeds_cfdml.loc[southern_edge]['wakedWindSpeed_at_8_m_per_s_at_180_deg'].plot(marker='o', ax=ax)
handles, labels = ax.get_legend_handles_labels()
ax.legend(handles, ['EV+LWF+BEET', 'CFD.ML'])
ax.set_title('Wake and blockage affected wind speeds at turbines\n Southern edge of the farm.\n Free stream: 8 m/s, 180 deg')
ax.set_ylabel('Wind speed [m/s]')
ax.set_xlabel('turbine label')

In [ ]:
fix, ax = plt.subplots(figsize=(7,4))
waked_windspeeds.loc[southern_edge]['wakedWindSpeed_at_8_m_per_s_at_90_deg'].plot(marker='o', ax=ax)
waked_windspeeds_cfdml.loc[southern_edge]['wakedWindSpeed_at_8_m_per_s_at_90_deg'].plot(marker='o', ax=ax)
handles, labels = ax.get_legend_handles_labels()
ax.legend(handles, ['EV+LWF+BEET', 'CFD.ML'])
ax.set_title('Wake and blockage affected wind speeds at turbines\n Southern edge of the farm.\n Free stream: 8 m/s, 90 deg')
ax.set_ylabel('Wind speed [m/s]')
ax.set_xlabel('turbine label')

### Pattern of production for a selected flowcase

Let's take a look at the production patterns predicted by both models and cross-correlate.

In [ ]:
# EV + LWF
spot_power = pd.DataFrame(result['windFarmAepOutputs'][0]['turbineFlowAndPerformanceMatricesWithMastBinning'])
spot_power.set_index('turbineName',drop=True, inplace=True)
spot_power['spotPower_at_8_m_per_s_at_180_deg'] = spot_power['spotPowerOutput_W'].map(lambda x: round(x[90][8],3))
spot_power['spotPower_at_8_m_per_s_at_90_deg'] = spot_power['spotPowerOutput_W'].map(lambda x: round(x[45][8],3))
# CFD.ML
spot_power_cfdml = pd.DataFrame(result_cfdml['windFarmAepOutputs'][0]['turbineFlowAndPerformanceMatricesWithMastBinning'])
spot_power_cfdml.set_index('turbineName',drop=True, inplace=True)
spot_power_cfdml['spotPower_at_8_m_per_s_at_180_deg'] = spot_power_cfdml['spotPowerOutput_W'].map(lambda x: x[90][8])
spot_power_cfdml['spotPower_at_8_m_per_s_at_90_deg'] = spot_power_cfdml['spotPowerOutput_W'].map(lambda x: x[45][8])

# Plot
spot_power['northing'] = per_turbine_results['northing']
spot_power['easting'] = per_turbine_results['easting']
spot_power_cfdml['northing'] = per_turbine_results['northing']
spot_power_cfdml['easting'] = per_turbine_results['easting']

result_no_interactions = json.loads(response_no_interactions.content)
spot_power_no_interactions = pd.DataFrame(result_no_interactions['windFarmAepOutputs'][0]['turbineFlowAndPerformanceMatricesWithMastBinning'])
spot_power_no_interactions.set_index('turbineName',drop=True, inplace=True)
spot_power_no_interactions['spotPower_at_8_m_per_s_at_180_deg'] = spot_power_no_interactions['spotPowerOutput_W'].map(lambda x: x[90][8])
scale = spot_power_no_interactions['spotPower_at_8_m_per_s_at_180_deg'].max()
fig, ax = plt.subplots(figsize=(10,4))
im = ax.scatter(spot_power['easting'], spot_power['northing'], c=spot_power['spotPower_at_8_m_per_s_at_180_deg'], cmap='BuGn', vmin=0.5*scale, vmax=scale)
ax.set_title("Pattern of production EV+LWF+BEET\n Free stream: 8 m/s, 180 deg")
ax.set_xlabel("Easting [m]")
ax.set_ylabel("Northing [m]")
ax.set_aspect('equal', adjustable='box')

fig2, ax2 = plt.subplots(figsize=(10,4))
im = ax2.scatter(spot_power_cfdml['easting'], spot_power_cfdml['northing'], c=spot_power_cfdml['spotPower_at_8_m_per_s_at_180_deg'], cmap='BuGn', vmin=0.5*scale, vmax=scale)
ax2.set_title("Pattern of production CFD.ML\n Free stream: 8 m/s, 180 deg")
ax2.set_xlabel("Easting [m]")
ax2.set_ylabel("Northing [m]")
ax2.set_aspect('equal', adjustable='box')

fig, ax = plt.subplots(figsize=(7,7))
ax.scatter(x=spot_power['spotPower_at_8_m_per_s_at_180_deg']/scale, y=spot_power_cfdml['spotPower_at_8_m_per_s_at_180_deg']/scale)
ax.set_title("Correlation of normalized power\nCFD.ML vs EV+LWF+BEET at The Bowl\nFree stream: 8 m/s, 180 deg")
ax.set_xlabel("EV+LWF+BEET")
ax.set_ylabel("CFD.ML")
ax.set_aspect('equal', adjustable='box')
ax.set_xlim((0.4,1))
ax.set_ylim((0.4,1))
ax.axline([0, 0], [1, 1])

The agreement isn't particularly good. 

Meanwhile we have strong evidence that CFD.ML replicates high-fidelity CFD modeling well at this site. 
So in the case of The Bowl, chances are high, that CFD.ML is closer to the truth. 

For more info see our recent [webinar](https://brandcentral.dnv.com/mars/embed?o=04B1B5EF7C529B02&c=10651&a=N), or presentation from WindEurope Tech 2023 Workshop in Lyon. 

Note: We drop the bowl from the CFD.ML training for the verification comparisson below.

![cfdml_replicates_cfd](images/cfdml_replicates_cfd.png)